# Slots
<img width=400 src="Images/Slot.png" />

Nun zu einem kurzen Kapitel über sogenannte Slots. Mit Hilfe dieser Konstruktion können wir die dynamische Erstellung von Klassenattributen verhindern. Wir hatten dies am Anfang des Kurses bereits als sehr unschöne Taktik beschrieben. Attribute sollten über die ```__init__()``` Methode zugeordnet werden, um Inkosistenzen zu vermeiden. Aber selbst in Klassen, deren Zugang über Properties und entsprechenden Schutz der Methoden gut abgesichert ist, kann Folgendes natürlich passieren:

In [44]:
class Myclass:

    def __init__(self, num):
        self.number=num #sofortige Umleitung in den setter!!
    
    @property #markiert number() als getter für __number
    def number(self): 
        #hier Code zum Manipulieren des Zugriffs. z.B. "Geben Sie Passwort ein"
        return self.__number
    
    @number.setter #markiert number() als setter für __number
    def number(self, number):        
        if number < 0:
            self.__number = 0
        elif number > 100:
            self.__number = 100
        else:
            self.__number = number

x= Myclass(110)  
print(x.number)
x.mein_neues_attribut="bla"
print(x.mein_neues_attribut)

100
bla


Im Dict der Instanz haben wir jetzt das Attribut ebenfalls eingetragen. 

In [45]:
x.__dict__

{'_Myclass__number': 100, 'mein_neues_attribut': 'bla'}

Wollen wir den Wert des Attributs sehen, geht dies auch so:

In [46]:
x.__dict__['mein_neues_attribut']

'bla'

Die dynamische Attributzuordnung ist aber, wie gezeigt,  nicht für alle Objekte möglich.

In [47]:
a=3
# a.anfangswert=5 #macht Fehler
my_list=[1,2,3]
# my_list.attr="Schöne Liste" #macht Fehler
Myclass.creator="Ich" # das geht
def foo():
    pass
foo.counter=0 #das geht
my_dict={}
# my_dict.attr=3 #macht Fehler

Das Verhindern des dynamischen Erzeugens von Attributen stellt einen bedeutenden Sicherheitsaspekt dar. Wie helfen hier slots?

In [48]:
class Person:
    __slots__ = 'name','nachname'
    def __init__(self, name):
        self.name = name
        # self.bla="bla" #macht fehler 'Person' object has no attribute 'bla'

inst=Person("Hubert")
print(inst.name)
inst.name = "Karl"
print(inst.name)
# inst.geb_datum="24.05.2000" #macht Fehler 'Person' object has no attribute 'geb_datum'
inst.nachname = "Müller" #kein Problem

Hubert
Karl


Wie wir sehen, legt ```__slots__``` am Beginn der Klasse fest, welches die erlaubten Attribute sind, nur diese können bearbeitet werden. Das gilt, wie oben zu sehen auch für die ```__init__()```. Das <b>Klassen-Dict</b> spiegelt den Zustand, daß wir jetzt slots verwendet haben. Wir finden darin unter ```__slots__``` die erlaubten Attribute aufgelistet. Auch für die Instanz können wir uns mit ```__slots``` die erlaubten Attribute ausgeben lassen. Ein dict hat aber die Instanz jetzt nicht mehr, konsequenterweise wird dies auch als Attribut angesehen und ist somit nicht erlaubt.

In [49]:
print(Person.__dict__)
print(100*"_")
print(inst.__slots__)
# print(inst.__dict__) #macht Fehler 'Person' object has no attribute '__dict__'


{'__module__': '__main__', '__slots__': ('name', 'nachname'), '__init__': <function Person.__init__ at 0x00000259F8F834C0>, 'nachname': <member 'nachname' of 'Person' objects>, 'name': <member 'name' of 'Person' objects>, '__doc__': None}
____________________________________________________________________________________________________
('name', 'nachname')


Was passiert bei Vererbung? Die Unterklasse kümmert sich nicht um die slots der Oberklasse!

In [50]:
class Animals:
    __slots__ = 'alter', 'art'
    def __init__(self, age, species):
        self.alter = age
        self.art = species

class Katzen(Animals):
    def __init__(self, age, name, meal):
        super(Katzen, self).__init__(age, name)
        self.meal = meal
        
me = Katzen(35, 'Violetta', 'Maus')
print(me.__dict__)
me.new_attr = "maumau"
print(me.__dict__)

{'meal': 'Maus'}
{'meal': 'Maus', 'new_attr': 'maumau'}


Umgekehrt, wenn nur die Unterklasse slots hat, genauso. Nur wenn beide ihre slots haben, kann man nicht mehr dynamische Attribute erzeugen.

In [51]:
class Animals:    
    def __init__(self, age, species):
        self.alter = age
        self.art = species

class Katzen(Animals):
    __slots__ = 'meal'
    def __init__(self, age, name, meal):
        super(Katzen, self).__init__(age, name)
        self.meal = meal
        
me = Katzen(35, 'Violetta', 'Maus')
print(me.__dict__)
me.new_attr = "maumau"
print(me.__dict__)

{'alter': 35, 'art': 'Violetta'}
{'alter': 35, 'art': 'Violetta', 'new_attr': 'maumau'}


Einige Schlussbemerkungen:<br>
Fraglos sind slots eine Hilfe zum Schutz der Klassen. Aber bei Klassen, die noch in Arbeit sind und viele Attribute haben, muß man die Verwaltung der slots ins Kalkül nehmen, die ja auch einen Aufwand darstellt.<br>
Ein bedeutender Vorteil bezüglich Ausführungsgeschwindigkeit und Speicherbedarf ist nicht zu erwarten.<br>
Der schlimmste Fehler für den User, der den Namen eines Attributs verwechseln könnte und dann das eigentlich gewünschte Attribut nicht angpasst , aber darüber auch keine Fehlermeldung bekommt, ist aber sicher vermieden:


In [52]:
#ohne Slots
class Person:
    
    def __init__(self, name):
        self.name = name
        # self.bla="bla" #macht fehler 'Person' object has no attribute 'bla'

inst=Person("Hubert")
print(inst.name)
inst.nname = "Karl"
print(inst.name)
# Schreibfehler mit nname, name wird nicht angepasst! Keine Fehlermeldung

Hubert
Hubert


In [53]:
#mit Slots
class Person:
    __slots__ = 'name','nachname'
    def __init__(self, name):
        self.name = name
        # self.bla="bla" #macht fehler 'Person' object has no attribute 'bla'

inst=Person("Hubert")
print(inst.name)
# inst.nname = "Karl" #macht Fehler 'Person' object has no attribute 'nname'


Hubert


Nun zu abstrakten Klassen und ihre Benutzung.